In [92]:
import pandas as pd
import numpy as np
import datetime 
import calendar
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", context="talk")
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import decomposition
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kartiktanksali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df_fake = pd.read_csv("Fake.csv")
df_real = pd.read_csv("True.csv")

In [3]:
df_fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [4]:
df_real

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [5]:
df_fake["RESULT"] = 0
df_real["RESULT"] = 1

In [6]:
df_fake

,title,text,subject,date,RESULT
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [7]:
df_real

,title,text,subject,date,RESULT
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [8]:
df = pd.concat([df_fake,df_real],ignore_index=True)

In [9]:
df

,title,text,subject,date,RESULT
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [10]:
df["Final_Text"] = df["title"] + df["text"]

In [18]:
df

,title,text,subject,date,RESULT,Final_Text
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0,Drunk Bragging Trump Staffer Started Russian ...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0,Sheriff David Clarke Becomes An Internet Joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0,Trump Is So Obsessed He Even Has Obama’s Name...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0,Pope Francis Just Called Out Donald Trump Dur...
...,...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1,'Fully committed' NATO backs new U.S. approach...
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1,LexisNexis withdrew two products from Chinese ...
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1,Minsk cultural hub becomes haven from authorit...
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1,Vatican upbeat on possibility of Pope Francis ...


In [11]:
df.dtypes

title         object
text          object
subject       object
date          object
RESULT         int64
Final_Text    object
dtype: object

In [12]:
df["subject"].value_counts()

politicsNews       11272
worldnews          10145
News                9050
politics            6841
left-news           4459
Government News     1570
US_News              783
Middle-east          778
Name: subject, dtype: int64

In [13]:
df["RESULT"].value_counts()

0    23481
1    21417
Name: RESULT, dtype: int64

In [14]:
df[['RESULT','subject','Final_Text']].groupby(['RESULT','subject']).count()

Final_Text
RESULT subject                    
0      Government News        1570
       Middle-east             778
       News                   9050
       US_News                 783
       left-news              4459
       politics               6841
1      politicsNews          11272
       worldnews             10145

In [ ]:
df.to_csv("Datset.csv")

In [5]:
df = pd.read_csv("Datset.csv")
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [62]:
lemmatizer = WordNetLemmatizer()
porterstem = PorterStemmer()
lst_lemmetized = []
lst_stemmed = []
def Preprocessing_Lemmatizing_Stemming(text):
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    words_lemmtized = [lemmatizer.lemmatize(w) for w in words]
    words_stemmed = [porterstem.stem(w) for w in words]
    lst_lemmetized.append(" ".join(words_lemmtized))
    lst_stemmed.append(" ".join(words_stemmed))

In [63]:
for i in df["Final_Text"]:
    Preprocessing_Lemmatizing_Stemming(str(i))

In [79]:
X = lst_lemmetized
y = df["RESULT"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)

pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', LogisticRegression())])

model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy of the model using Lemmetized text {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print(confusion_matrix(y_test,y_pred))


X = lst_stemmed
y = df["RESULT"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)


pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', LogisticRegression())])

model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy of the model using Stemmed text {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print(confusion_matrix(y_test,y_pred))

/Users/kartiktanksali/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of the model using Lemmetized text 98.49%
[[6919  106]
 [  97 6348]]


/Users/kartiktanksali/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of the model using Stemmed text 98.39%
[[6907  118]
 [  99 6346]]


In [81]:
X = lst_lemmetized
y = df["RESULT"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)

pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', MultinomialNB())])

model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy of the model using Lemmetized text {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print(confusion_matrix(y_test,y_pred))


X = lst_stemmed
y = df["RESULT"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)


pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', MultinomialNB())])

model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy of the model using Stemmed text {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print(confusion_matrix(y_test,y_pred))

Accuracy of the model using Lemmetized text 93.9%
[[6646  379]
 [ 443 6002]]
Accuracy of the model using Stemmed text 93.5%
[[6637  388]
 [ 487 5958]]


In [86]:
X = lst_lemmetized
y = df["RESULT"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)

pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', KNeighborsClassifier())])

model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy of the model using Lemmetized text {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print(confusion_matrix(y_test,y_pred))


X = lst_stemmed
y = df["RESULT"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)


pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', KNeighborsClassifier())])

model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy of the model using Stemmed text {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print(confusion_matrix(y_test,y_pred))

Accuracy of the model using Lemmetized text 85.82%
[[5490 1535]
 [ 375 6070]]
Accuracy of the model using Stemmed text 85.57%
[[5452 1573]
 [ 371 6074]]


In [87]:
X = lst_lemmetized
y = df["RESULT"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)

pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', RandomForestClassifier())])

model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy of the model using Lemmetized text {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print(confusion_matrix(y_test,y_pred))


X = lst_stemmed
y = df["RESULT"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)


pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', RandomForestClassifier())])

model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy of the model using Stemmed text {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print(confusion_matrix(y_test,y_pred))

/Users/kartiktanksali/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy of the model using Lemmetized text 97.39%
[[6936   89]
 [ 263 6182]]


/Users/kartiktanksali/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy of the model using Stemmed text 95.43%
[[6862  163]
 [ 452 5993]]


In [88]:
X = lst_lemmetized
y = df["RESULT"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)

pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', DecisionTreeClassifier())])

model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy of the model using Lemmetized text {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print(confusion_matrix(y_test,y_pred))


X = lst_stemmed
y = df["RESULT"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)


pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', DecisionTreeClassifier())])

model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy of the model using Stemmed text {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print(confusion_matrix(y_test,y_pred))

Accuracy of the model using Lemmetized text 99.71%
[[7008   17]
 [  22 6423]]
Accuracy of the model using Stemmed text 99.64%
[[7001   24]
 [  25 6420]]


In [106]:
X = lst_lemmetized
y = df["RESULT"]



pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', DecisionTreeClassifier())])

criterion = ['gini', 'entropy']
max_depth = [2,4,6,8]

parameters = dict(model__criterion=criterion, model__max_depth=max_depth)

clf = GridSearchCV(pipe, parameters)

clf.fit(X, y)

/Users/kartiktanksali/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                    

In [108]:
print("Grid Search results")
print('Criterion:', clf.best_estimator_.get_params()['model__criterion'])
print('Max_depth:', clf.best_estimator_.get_params()['model__max_depth'])

Grid Search results
Criterion: entropy
Max_depth: 2
